# Financial Backtesting with q / KDB-X

## What is this notebook about?

This notebook demonstrates how to build a **financial backtesting system** using KDB-X/q. We'll simulate a simple trading strategy and calculate how much profit or loss it would have generated.

## Why use KDB-X/q for backtesting?

KDB-X/q is a high-performance database and programming language specifically designed for time-series data. It's used by major financial institutions because:

- **Speed**: Processes millions of trades in milliseconds
- **Time-series operations**: Built-in functions like `asof joins` make matching trades to market prices trivial
- **Columnar storage**: Extremely efficient for financial data analysis
- **Concise syntax**: Complex operations can be expressed in just a few lines

## Prerequisites
- Requires KDB-X to be installed, you can sign up and download on [Developer Center](https://developer.kx.com/products/kdb-x/install). For full install instructions see: [KDB-X Install](https://code.kx.com/kdb-x/).

- To Install KDB-X Python: `pip install --upgrade --pre pykx` (This is required to run q in a Python notebook, but you can run the q directly from a q session if you prefer.)

### What is PyKX?

PyKX is a Python library that lets us run q/KDB-X code directly from Python. In this notebook, we use the `%%q` magic command to write q code in Jupyter cells, combining the interactivity of notebooks with the power of q. You can run these commands directly in a q session if preferred. 

In [1]:
import pykx as kx

## Step 1: Generate Simulated Market Data

Before we can backtest a trading strategy, we need market data. In the real world, this would come from historical databases, but for this example we'll generate realistic-looking random data.

### What we're creating:
- **Trades** for three stocks: AAPL (Apple), GOOG (Google), IBM
- **Date range**: January 1-31, 2015
- **Fields**: date, time, symbol, quantity, price, market cap

### Why this matters:
In real backtesting, you need to know the **exact price** at which you could have executed trades. The `aj` (asof join) function in q makes this easy - it finds the most recent price as of a given timestamp, which is exactly what happens in real trading.

### The Power of q:
Notice how we can:
1. Generate 10,000 random trades in one line
2. Sort by multiple columns (`dt` and `tm`) with a simple expression
3. Update prices conditionally by symbol with clean syntax
4. Calculate VWAP (volume-weighted average price) in a single operation

This type of data manipulation would require many more lines in traditional languages.

In [2]:
%%q

/function outputting sample market data
mktrades:{[tickers; sz]
  dt:2015.01.01+sz?31;              / Generate random dates in January 2015
  tm:sz?24:00:00.000;               / Generate random times throughout the day
  sym:sz?tickers;                   / Randomly assign stock symbols
  qty:10*1+sz?1000;                 / Random quantities (10 to 10,000 shares)
  px:90.0+(sz?2001)%100;            / Random prices around $90-110
  mc: 10000;                        / Market cap (simplified, constant)
  t:([] dt; tm; sym; qty; px; mc);  / Create table with these columns
  t:`dt`tm xasc t;                  / Sort by date then time (chronological order)
  t:update px:6*px from t where sym=`goog;  / GOOG trades at ~6x the base price
  t:update px:2*px from t where sym=`ibm;   / IBM trades at ~2x the base price
  t};

/running function to generate sample market data  
trades:mktrades[`aapl`goog`ibm; 10000];
trades: update vwapx: qty wavg px by dt,sym from trades;  / Calculate VWAP per day per symbol

/sample strategy - buy goog, appl, ibm on 1/1/15, sell on 1/31/15
example:select distinct sym,size:1000,tradein:first dt,tradeout:last dt from trades;

/create backtest table
bt_table:
 select
    sym,
    size,
    tradein,
    tradeout
 from
    example
 where
    sym in `ibm`goog;


/----
/example 1: basic backtest logic
/trades = tick data
/bt_table = backtest table
t1:aj[`sym`dt;select sym,size,dt:tradein from bt_table;trades]; /takes last trade for IBM on 1/1/15 as entry price
t2:aj[`sym`dt;select sym,size,dt:tradeout from bt_table;trades]; /takes last trade for IBM on 1/31/15 as exit price
(t2[`px]-t1[`px])*t1[`size]; /pnl

/----
/example 2: backtest function (simplest form, no timeseries nuance)
bt:{[table] /function takes in one parameter, the backtest table
    /find entry price for each sym via AJ
    tentry:aj[`sym`dt;select sym,size,dt:tradein from table;trades];
    /find exit price for each sym via AJ
    texit:aj[`sym`dt;select sym,size,dt:tradeout from table;trades];
    /calculate pnl for each sym over trade lifetime
    pnl_by_sym: (texit[`px]-tentry[`px])*tentry[`size];
    pnl: sum pnl_by_sym;
    /return pnl
    pnl
 }
 
/run example backtest function on input table to derive pnl
res:bt[bt_table]

/----
/example 3: backtest function highlighting PnL at each trading day using vwap)
btn:{[table]
    /generate dates between entry and exit
    daterack:flip exec dt:{y+til x--1+y}[max tradeout;min tradein] from table;
    daterack:daterack cross distinct select sym,size from bt_table;

    /calc vwap by date from trades
    tvwap: select vwap:qty wavg px by dt,sym from trades;
    tvwap: () xkey tvwap;
    /use aj to find sym's price as of each date from the trades marketdata table (here, vwap proxy for each date)
    /daterack- dt,sym,size
    /trades- `dt`tm`sym`qty`px`mc
    daterack:aj[`dt`sym;daterack;select from tvwap];
    /pnl calc
    daterack:update pnl: size*vwap-first[vwap] by sym from daterack;
    `pnlseries`summary`pctreturn!(
    ([] dt: asc distinct exec dt from daterack;pnl: exec pnl from select sum pnl by dt from daterack);
    last daterack[`pnl];
    (last daterack[`pnl])%(bt_table[0;`size]*first daterack[`vwap])
    )
 };
 
/run backtest
btn[bt_table]

pnlseries| +`dt`pnl!(`s#2015.01.01 2015.01.02 2015.01.03 2015.01.04 2015.01.0..
summary  | 1495.285
pctreturn| 0.002510236


## Step 2: Define Our Trading Strategy

Now we need to define **what** we want to backtest. Our strategy is simple:

### The Strategy:
- **Buy** 1,000 shares each of GOOG and IBM on January 1, 2015
- **Hold** them for the entire month
- **Sell** everything on January 31, 2015

### The Backtest Table:
The `bt_table` contains our planned trades:
- `sym`: which stock to trade
- `size`: how many shares (1,000 each)
- `tradein`: when to buy (entry date)
- `tradeout`: when to sell (exit date)

This is a "buy and hold" strategy - we're testing whether simply holding these two tech stocks for a month would have been profitable.

## Example 1: Basic Backtest Logic

This is the simplest way to calculate profit/loss. We need to answer two questions:
1. **What price did we buy at?** (entry price on Jan 1)
2. **What price did we sell at?** (exit price on Jan 31)

### The Magic of Asof Join (`aj`):

The `aj` function is one of q's superpowers. It finds the **most recent** matching record as of a specific time. This is exactly what you need for financial data:

```q
aj[`sym`dt; our_trades; market_data]
```

This says: "For each of our trades, find the last market price available on that date for that symbol."

### Why this is powerful:
- In traditional SQL, this would require complex subqueries or window functions
- In pandas, you'd need to merge, sort, and use groupby operations
- In q, it's **one function call** and it's blazingly fast

### The Calculation:
```
PnL = (Exit Price - Entry Price) × Number of Shares
```

Let's see it in action:

In [3]:
%%q
bt[bt_table]

16600f


### Result: $16,600 profit

Our simple strategy would have made $16,600 over the month. But this is just a single number - we don't know:
- How the PnL changed day by day
- Whether we were ever losing money during the month
- What the percentage return was

Let's dig deeper with the next example...

## Example 2: Advanced Backtest with Daily PnL

The `btn` function (backtest with nuance) gives us much more insight. Instead of just final PnL, we get:

### What this calculates:
1. **Daily PnL**: How much profit/loss we had at the end of each day
2. **PnL Series**: A complete timeseries showing portfolio value over time
3. **Percent Return**: What percentage of our initial investment we made

### How it works:
1. **Generate all dates** between entry and exit
2. For each date, calculate the VWAP (volume-weighted average price)
3. Use `aj` to find each stock's VWAP on each day
4. Calculate daily PnL relative to our entry price

### Why VWAP?
VWAP (Volume-Weighted Average Price) is a more realistic price than just using the last trade of the day. It represents the average price weighted by trading volume - essentially the "fair" price for that day.

### The q Advantage:
Look at this line:
```q
tvwap: select vwap:qty wavg px by dt,sym from trades
```

This calculates volume-weighted average prices grouped by date and symbol in **one line**. In other languages, this would require:
- Grouping data
- Calculating weighted averages manually
- Aggregating results
- Multiple lines of code

Let's run it:

In [4]:
%%q
\c 1000 1000
btn[bt_table]

pnlseries| +`dt`pnl!(`s#2015.01.01 2015.01.02 2015.01.03 2015.01.04 2015.01.05 2015.01.06 2015.01.07 2015.01.08 2015.01.09 2015.01.10 2015.01.11 2015.01.12 2015.01.13 2015.01.14 2015.01.15 2015.01.16 2015.01.17 2015.01.18 2015.01.19 2015.01.20 2015.01.21 2015.01.22 2015.01.23 2015.01.24 2015.01.25 2015.01.26 2015.01.27 2015.01.28 2015.01.29 2015.01.30 2015.01.31;0 4091.652 4464.118 10419.29 1920.787 7759.533 -2627.317 8466.34 1303.098 13082 9628.353 4784.923 436.9293 4655.67 8299.684 -3836.032 5400.844 9045.004 7603.171 1681.742 2462.197 1968.41 4752.998 -474.5213 8861.784 460.883 1926.882 7370.895 5069.713 5854.743 5480.873)
summary  | 1495.285
pctreturn| 0.002510236


## Understanding the Results

### Key Findings:

1. **Summary PnL: $1,495.28**
   - This is our final profit using VWAP prices (more realistic than the simple $16,600 using last trades)
   - The difference shows why using VWAP matters for accurate backtesting

2. **Percent Return: 0.25%**
   - Our investment returned 0.25% over the month
   - This is modest but positive

3. **Daily PnL Series:**
   - We can see the portfolio value changed every day
   - Notice **January 7th and 16th** showed losses (negative PnL)
   - **January 10th** was our best day with ~$13,000 gain
   - The portfolio was volatile but generally trended upward

### What This Tells Us:
- The strategy wasn't consistently profitable day-to-day
- We had drawdown periods (times when we were losing money)
- Risk management would have been important
- A trader might have panicked and sold on Jan 7th or 16th when showing losses

### The Power of q for This Analysis:
We generated:
- 10,000 simulated trades
- Daily VWAP calculations
- 31 days of portfolio valuations
- Complete risk metrics

All in **just a few lines of q code** and executed in **milliseconds**. This same analysis in traditional tools would require significantly more code and computation time.

## A Detailed Look at Entry and Exit Prices

Let's examine exactly what prices we entered and exited at for each stock:

In [5]:
%%q


t1

(::;::)


t2


(::;::)

sum (t2[`px]-t1[`px])*t1[`size]



sym  size dt         tm           qty  px     mc    vwapx   
------------------------------------------------------------
goog 1000 2015.01.01 23:15:36.077 8520 590.7  10000 595.6753
ibm  1000 2015.01.01 23:35:57.076 6530 207.16 10000 200.2504
::
::
sym  size dt         tm           qty  px     mc    vwapx   
------------------------------------------------------------
goog 1000 2015.01.31 23:53:43.752 7780 632.76 10000 599.6609
ibm  1000 2015.01.31 23:59:07.179 1040 181.7  10000 201.7456
::
::
16600f


## Breaking Down the Trades

### Entry Positions (January 1st):
- **GOOG**: Bought 1,000 shares at $590.70
  - Last trade of the day at 11:15 PM
  - Total investment: $590,700
  
- **IBM**: Bought 1,000 shares at $207.16
  - Last trade of the day at 11:35 PM  
  - Total investment: $207,160

**Total Initial Investment: $797,860**

### Exit Positions (January 31st):
- **GOOG**: Sold 1,000 shares at $632.76
  - Last trade of the month at 11:53 PM
  - Total proceeds: $632,760
  - **Profit on GOOG: $42,060** ✓
  
- **IBM**: Sold 1,000 shares at $181.70
  - Last trade of the month at 11:59 PM
  - Total proceeds: $181,700  
  - **Loss on IBM: -$25,460** ✗

### The Final Math:
```
GOOG: ($632.76 - $590.70) × 1,000 = +$42,060
IBM:  ($181.70 - $207.16) × 1,000 = -$25,460
                           Total    = +$16,600
```

### Key Insight:
While we made money overall, **IBM actually lost money** over the month. GOOG's strong performance (+7.1%) more than compensated for IBM's decline (-12.3%). This is why diversification matters - and why seeing individual stock performance is important in backtesting!

---

## Conclusion: Why KDB-X/q for Backtesting?

This notebook demonstrated several key advantages:

1. **Time-series operations are natural**: The `aj` function makes it trivial to match trades to prices
2. **Concise and readable**: Complex calculations in just a few lines
3. **Fast execution**: Processing 10,000 trades with daily aggregations in milliseconds
4. **Built for finance**: Features like VWAP, grouping by symbol/date, and temporal joins are first-class operations

For professional backtesting with millions of trades, real-time data feeds, or complex multi-asset strategies, KDB-X/q's performance advantages become even more pronounced. What we did here in a simple notebook would scale to institutional-grade backtesting infrastructure with minimal code changes.